# Virtual Neuron example

In this example we set up a larger network of neurons, then the network is modified so that only a few core neurons remain normal, while the rest of the population is changed to be virtual neurons. Virtual neurons are not simulated, instead their activity is represented by data read from file. All the synapses in the network remain though. This allows the user to simulate a subset of the network, but retaining the full input those neurons should have received if all the neurons had been simulated normally.

In [1]:
import os

number_of_neurons = 1000
num_sim_neurons = 10
network_path = os.path.join("networks","virtual_network_example")
modified_network_file = os.path.join(network_path, "modified-network.hdf5")

%matplotlib inline

## Start ipcluster

For larger networks it can be helpful to run the network creation code in parallel. You can start the ipyparallel workers using ```start_ipcluster.sh``` located in the ```examples/notebook``` directory. After starting ipcluster give it a few seconds to startup before proceeding with the rest of the commands in this notebook. To stop the cluster use ```ipcluster stop```.

For instructions how to install and activate ipyparallel for your notebook see https://github.com/ipython/ipyparallel

If you just want to run the notebook in serial, then you do not have to do anything extra. The code automatically detects if ipyparallel is running.

## Check if ipyparallel is running and set ```SNUDDA_DATA```

In [2]:
from ipyparallel import Client
import snudda

os.environ["IPYTHONDIR"] = ".ipython"
os.environ["IPYTHON_PROFILE"] = "default"

u_file = os.path.join(".ipython", "profile_default", "security", "ipcontroller-client.json")
if os.path.exists(u_file):
    rc = Client(url_file=u_file, timeout=120, debug=False)
else:
    print("Warning, running in serial.")
    rc = None

# Check if the user has the updated BasalGangliaData installed, if not use the old data provided
bg_path = "../../../../BasalGangliaData/data/"
if os.path.isdir(bg_path):
    snudda_data = bg_path
else:
    snudda_data = os.path.join(os.path.dirname(snudda.__file__), "data")

Warning, running in serial.


In [3]:
import os
from snudda import SnuddaInit

si = SnuddaInit(network_path=network_path, random_seed=12345, snudda_data="/home/hjorth/HBP/BasalGangliaData/data/")
si.define_striatum(num_dSPN=500, num_iSPN=500, num_FS=0, num_LTS=0, num_ChIN=0,
                    volume_type="cube", neurons_dir="$DATA/neurons")
si.write_json()

Using cube for striatum
Neurons for striatum read from /home/hjorth/HBP/BasalGangliaData/data/neurons/striatum
Adding neurons: dSPN from dir /home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/dspn
Adding neurons: iSPN from dir /home/hjorth/HBP/BasalGangliaData/data/neurons/striatum/ispn
Writing networks/virtual_network_example/network-config.json


In [4]:
from snudda import SnuddaPlace
sp = SnuddaPlace(network_path=network_path)
sp.place()

Reading SNUDDA_DATA=/home/hjorth/HBP/BasalGangliaData/data/ from networks/virtual_network_example/network-config.json


In [5]:
from snudda import SnuddaDetect

sd = SnuddaDetect(network_path=network_path, rc=rc)
sd.detect()

Reading SNUDDA_DATA=/home/hjorth/HBP/BasalGangliaData/data/ from networks/virtual_network_example/network-config.json
No d_view specified, running distribute neurons in serial
Processing hyper voxel : 42/150 (951 neurons)
Processing hyper voxel : 41/150 (943 neurons)
Processing hyper voxel : 36/150 (925 neurons)
Processing hyper voxel : 37/150 (924 neurons)
Processing hyper voxel : 72/150 (863 neurons)
Processing hyper voxel : 71/150 (840 neurons)
Processing hyper voxel : 67/150 (830 neurons)
Processing hyper voxel : 66/150 (808 neurons)
Processing hyper voxel : 11/150 (213 neurons)
Processing hyper voxel : 12/150 (213 neurons)
Processing hyper voxel : 7/150 (198 neurons)
Processing hyper voxel : 6/150 (194 neurons)
Processing hyper voxel : 47/150 (170 neurons)
Processing hyper voxel : 46/150 (164 neurons)
Processing hyper voxel : 43/150 (161 neurons)
Processing hyper voxel : 38/150 (152 neurons)
Processing hyper voxel : 73/150 (128 neurons)
Processing hyper voxel : 76/150 (124 neurons

In [6]:
from snudda import SnuddaPrune

sp = SnuddaPrune(network_path=network_path, rc=rc)
sp.prune()

Worker synapses: 22/6138360 (heap size: 30)
Worker synapses: 4229809/6138360 (heap size: 24)
Worker synapses: 6138360/6138360 (heap size: 0)
Read 6138360 out of total 6138360 synapses


In [7]:
if number_of_neurons <= 100:
    # Do not plot too large networks, as that will take a really long time
    from snudda.plotting import PlotNetwork
    pn = PlotNetwork(network_path)
    pn.plot()
    pn.close()

# Converting neurons to virtual neurons

We want to simulate the centre neurons, all other neurons are changed into virtual neurons.

In [8]:
from snudda.utils.ablate_network import SnuddaAblateNetwork
from snudda.utils import SnuddaLoad

sa = SnuddaAblateNetwork(network_file=network_path)
sl = sa.snudda_load
virtual_idx = sorted(list(set(sl.iter_neuron_id()) - set(sl.get_centre_neurons_iterator(n_neurons=num_sim_neurons))))

sa.make_virtual(virtual_idx)
sa.write_network(modified_network_file)

Loading 1000 neurons with 325565 synapses and 0 gap junctions
Writing to networks/virtual_network_example/modified-network.hdf5
Copying morphologies
Keeping 1000 neurons.
Making neuron id 0 (dSPN_0) virtual (old ID 0)
Making neuron id 1 (dSPN_0) virtual (old ID 1)
Making neuron id 2 (dSPN_0) virtual (old ID 2)
Making neuron id 3 (dSPN_1) virtual (old ID 3)
Making neuron id 4 (dSPN_2) virtual (old ID 4)
Making neuron id 5 (dSPN_3) virtual (old ID 5)
Making neuron id 6 (dSPN_3) virtual (old ID 6)
Making neuron id 7 (iSPN_1) virtual (old ID 7)
Making neuron id 8 (iSPN_1) virtual (old ID 8)
Making neuron id 9 (iSPN_2) virtual (old ID 9)
Making neuron id 10 (iSPN_3) virtual (old ID 10)
Making neuron id 11 (dSPN_0) virtual (old ID 11)
Making neuron id 12 (dSPN_1) virtual (old ID 12)
Making neuron id 13 (dSPN_1) virtual (old ID 13)
Making neuron id 14 (iSPN_0) virtual (old ID 14)
Making neuron id 15 (iSPN_0) virtual (old ID 15)
Making neuron id 16 (iSPN_0) virtual (old ID 16)
Making neuron id

# Generate the input for the neurons, and the activity of the virtual neurons

In [9]:
# TODO: TO BE CONTINUED, WE NEED TO UPDATE HOW INPUT IS SET FOR VIRTUAL NEURONS -- allow loading from file, etc